<a href="https://colab.research.google.com/github/mplaza27/Image-Classification/blob/main/Image_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
from scipy.optimize import curve_fit
import torch
import torchvision
import numpy as np
import torch.utils.data as data
import torch.nn.utils.prune as prune

#Constants

torch.set_printoptions(threshold=100000000000)  # For debugging

# Setting Seed
#np.random.seed(9)

DEVICE = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)


#import MNIST dataset
def import_training_data() -> tuple[data.Dataset,data.Dataset]:

  training_data = torchvision.datasets.MNIST(
    root="data",
    train=True,
    download=True,
    transform=torchvision.transforms.ToTensor()
  )

  test_data = torchvision.datasets.MNIST(
    root="data",
    train=False,
    download=True,
    transform=torchvision.transforms.ToTensor()
  )

  return (training_data, test_data)

def count_zeros(tensor):
  print(28*28*10 - torch.count_nonzero(tensor))

def inv_fit(x,y):
    def inv_func(x, a, b, c):
        return (a / (x-b)) + c
    popt, _ = curve_fit(inv_func, x, y, p0 = (10,-10,0.5))
    return popt, inv_func

class Neural_Network(torch.nn.Module):

  # Constructor
  def __init__(self, b_size=100, l_rate=1, percentage=0, directory='file.csv'):
    super().__init__()

    self.file = open(directory, "w")
    INPUT_SIZE = 28 * 28
    OUTPUT_SIZE = 10
    self.linlayer = torch.nn.Linear(INPUT_SIZE, OUTPUT_SIZE, bias=False)
    self.model = torch.nn.Sequential(self.linlayer)
    self.criterion = torch.nn.CrossEntropyLoss()
    self.optimizer = torch.optim.Adadelta(self.model.parameters(), lr=l_rate)
    self.b_size = b_size
    self.percentage = percentage
    self.flatten = torch.nn.Flatten()
    self.clear()

    self.ce_asy_list = [0]
    self.a_list = [0]
    self.b_list = [0]
    self.stop_counter = 0
    self.ces = []
    self.stop_training = False


  # Manually initialize weight matrices to ones
  def clear(self) -> None:
    with torch.no_grad():
      for _, param in self.model.named_parameters():
        if param.requires_grad:
          param.copy_(torch.ones_like(param))

  def forward(self, x):
    return(self.model(self.flatten(x)))

def train(dataloader, testloader, model, epoch) -> None:
    size = len(dataloader.dataset)

    if (epoch ==1):
        model.file.write("0\t[ 0000/60000]\t--\t") # Make sure that avg CE is the same as this.
        test(testloader, model)

    model.train()

    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(DEVICE), y.to(DEVICE)

        # Compute prediction error
        pred = model(X)
        loss = model.criterion(pred, y)

        # Backpropagation
        model.optimizer.zero_grad()
        loss.backward()

        model.optimizer.step()

        pred = model(X)                             # These two lines ensure that the
        loss = model.criterion(pred, y)             # Loss after the backwards pass is printed
        model.ces.append(loss.detach().numpy())

        loss, current = loss.item(), (batch+1) * len(X)
        model.file.write(f"{epoch}\t[{current:>5d}/{size:>5d}]\t{loss:>7f}")

        if len(model.ces) >= (1000*256/model.b_size) + 50:
            model.stop_training = True
            break
        try:
            params, inv_func = inv_fit(range(len(model.ces)), np.array(model.ces))
            a, b, c = params
            model.ce_asy_list.append(c)
            model.a_list.append(a)
            model.b_list.append(b)

            if 0.99 * model.ce_asy_list[-2] < model.ce_asy_list[-1] < 1.01 * model.ce_asy_list[-2]:
                    model.stop_counter += 1
                    if model.stop_counter > 4:
                        print('stop training (fit)')
                        model.stop_training = True
                        break
            else:
              model.stop_counter = 0

        except (RuntimeError, TypeError) as error:
            model.ce_asy_list.append(0)
            model.a_list.append(0)
            model.b_list.append(0)

        test(testloader,model)

def test(dataloader, model) -> None:
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0.0, 0.0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(DEVICE), y.to(DEVICE)
            pred = model(X)
            test_loss += model.criterion(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    test.counter +=1 # line counter
    model.file.write(f"\t{(100*correct):>0.3f}\t\t{test_loss:>8f}\t{test.counter}\t\t{model.ce_asy_list[-1]:>8f}\t{model.a_list[-1]:>8f}\t{model.b_list[-1]:>8f}\n")
    print(test.counter)

(train_dataset, test_dataset) = import_training_data() # Downloads data
def main(percentage,bs,lr,lr_dir) -> None:
  test.counter = -1

  m = Neural_Network(bs,lr,percentage,lr_dir).to(DEVICE)
  m.file.write("Epoch\t[current/size]\tCE\t\tAccuracy(%)\tAVG_CE\t\tBatch\t\tCE_ASY\t\tA\t\tB\n") #Display title

  train_loader = data.DataLoader(train_dataset, batch_size=m.b_size, shuffle=True)
  test_loader = data.DataLoader(test_dataset, batch_size=m.b_size, shuffle=False)

  PERCENT = m.percentage

  prune.random_unstructured(m.linlayer, name="weight", amount=int((PERCENT/100.0) * 7840))

  i = 1
  while not m.stop_training:
    print(i)
    train(train_loader,test_loader,m,i)
    i = i + 1

  count_zeros(m.linlayer.weight.data)
  m.file.close()

ACCEPTABLE_BATCH_SIZES = [256, 1000, 2000, 4000, 6000, 12000, 15000, 30000, 60000,1,8,32]
ACCEPTABLE_LR = [1, 0.1, 0.5, 2, 3, 4, 5, 10]
#ACCEPTABLE_LR = [1, 0.01,0.1, 0.5, 2, 3, 4, 5, 10,20] Removed 0.01 and 20 to save time, and because they seem unreasonable.
PERCENTAGES = range(0,5)

def create_directories():
    for percentage in PERCENTAGES:
        percentage_dir = f"percentage_{percentage}"
        os.makedirs(percentage_dir, exist_ok=True)
        for batch_size in ACCEPTABLE_BATCH_SIZES:
            batch_size_dir = os.path.join(percentage_dir, f"batch_size_{batch_size}")
            os.makedirs(batch_size_dir, exist_ok=True)
            for lr in ACCEPTABLE_LR:
                lr_dir = os.path.join(batch_size_dir, f"lr_{lr}")
                os.makedirs(lr_dir, exist_ok=True)

    call_main_round_robin()

def call_main_round_robin():
    total_iterations = 100
    iterations_done = 0
    lr_index = 0
    runs = 0

    while iterations_done < total_iterations:
        for percentage in PERCENTAGES:
            for batch_size in ACCEPTABLE_BATCH_SIZES:
                for lr in ACCEPTABLE_LR:
                    # Get the corresponding lr directory
                    filename = f"P%-{percentage}_BS-{batch_size}_LR-{lr}_run-{runs}"
                    lr_dir = os.path.join(f"percentage_{percentage}", f"batch_size_{batch_size}", f"lr_{ACCEPTABLE_LR[lr_index]}",filename)

                    main(percentage, batch_size, ACCEPTABLE_LR[lr_index], lr_dir)  # Call main with the current lr
                    iterations_done += 1
                    lr_index = (lr_index + 1) % len(ACCEPTABLE_LR)  # Move to the next lr in a round-robin fashion
        runs = runs + 1

create_directories()


100%|██████████| 9912422/9912422 [00:00<00:00, 85586882.83it/s]


Extracting data/MNIST/raw/train-images-idx3-ubyte.gz to data/MNIST/raw



100%|██████████| 28881/28881 [00:00<00:00, 141348534.22it/s]


Extracting data/MNIST/raw/train-labels-idx1-ubyte.gz to data/MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 26358001.08it/s]


Extracting data/MNIST/raw/t10k-images-idx3-ubyte.gz to data/MNIST/raw



100%|██████████| 4542/4542 [00:00<00:00, 5968210.77it/s]


Extracting data/MNIST/raw/t10k-labels-idx1-ubyte.gz to data/MNIST/raw

1
0
1
2
3
4
5
